# Usermore soap case study

Guilherme Fernandes e Lauro Solia

In [1]:
from pprint import pprint

import gurobipy
from gurobipy import Model, GRB

import pandas as pd

## 1. Defining the sets

In [2]:
plants = [
    "Covington, KY",
    "New York, NY",
    "Arlington, TX",
    "Long Beach, CA",
]

existing_warehouses = [
    "Atlanta",
    "Boston",
    "Buffalo",
    "Chicago",
    "Cleveland",
    "Davenport",
    "Detroit",
    "Grand Rapids",
    "Greensboro",
    "Kansas City",
    "Baltimore",
    "Memphis",
    "Milwaukee",
    "Orlando",
    "Pittsburgh",
    "Portland",
    "W Sacramento",
    "W Chester",
]

potential_warehouses = [
    "Albuquerque",
    "Billings",
    "Denver",
    "El Paso",
    "Camp Hill",
    "Houston",
    "Las Vegas",
    "Minneapolis",
    "New Orleans",
    "Phoenix",
    "Richmond",
    "St Louis",
    "Salt Lake City",
    "San Antonio",
    "Seattle",
    "Spokane",
    "San Francisco",
    "Indianapolis",
    "Louisville",
    "Columbus",
    "New York",
    "Hartford",
    "Miami",
    "Mobile",
    "Memphis *",
    "Chicago *",
]

# Merge existing + potential
warehouses = existing_warehouses + potential_warehouses

In [3]:
print("Number of plants: ", len(plants))
print("Number of existing warehouses: ", len(existing_warehouses))
print("Number of potential warehouses: ", len(potential_warehouses))
print("Number of total warehouses: ", len(warehouses))
print("Obs.: Each plant is also an existing warehouse")

Number of plants:  4
Number of existing warehouses:  18
Number of potential warehouses:  26
Number of total warehouses:  44
Obs.: Each plant is also an existing warehouse


## 2. Parameters

In [4]:
# read state demands from Figure 1
state_demand = {
    # West region:
    "WA": 32437,  # Washington
    "OR": 31365,  # Oregon
    "CA": 135_116,  # California
    "NV": 16755,  # Nevada
    "AZ": 9063,  # Arizona
    "ID": 7153,  # Idaho
    # Northwest region:
    "UT": 9001,  # Utah
    "MT": 4140,  # Montana
    "ND": 5703,  # North Dakota
    "WY": 1004,  # Wyoming
    "CO": 11147,  # Colorado
    "SD": 1049,  # South Dakota
    "NE": 7347,  # Nebraska
    "KS": 6961,  # Kansas
    "MN": 5633,  # Minnesota
    "IA": 32175,  # Iowa
    "MO": 41680,  # Missouri
    # Southwest region:
    "NM": 3536,  # New Mexico
    "TX": 80438,  # Texas
    "OK": 13517,  # Oklahoma
    "AR": 4910,  # Arkansas
    "LA": 15011,  # Louisiana
    # Midwest region:
    "WI": 37448,  # Wisconsin
    "IL": 72839,  # Illinois
    "MI": 105_181,  # Michigan
    "IN": 43994,  # Indiana
    "KY": 3870,  # Kentucky
    "OH": 155_123,  # Ohio
    # Northeast region:
    "ME": 15829,  # Maine
    "NH": 4546,  # New Hampshire
    "RI": 17000,  # Rhode Island
    "NJ": 21154,  # New Jersey
    "NY": 160_917,  # New York
    "PA": 65108,  # Pennsylvania
    "CT": 26_187,  # Connecticut
    "MA": 37087,  # Massachusetts
    "VA": 17667,  # Virginia
    "WV": 9168,  # West Virginia
    "MD": 19284,  # Maryland
    "VT": 2928,  # Vermont
    "DE": 3044,  # Delaware
    # Southeast region:
    "TN": 42479,  # Tennessee
    "MS": 15_205,  # Mississippi
    "AL": 15835,  # Alabama
    "GA": 29559,  # Georgia
    "FL": 46405,  # Florida
    "SC": 5680,  # South Carolina
    "NC": 28_348,  # North Carolina
}
S = state_demand  # Sales

print("Total states number: ", len(S))
print("Total demand: ", sum(S.values()))
print("Each state represents a demand center")

# TODO: falta alguém ler e verificar os dados (o input foi feito de forma manual)

# NOTE: Valor total deve ser 1_477_026, então precisa ajustar NY (que nao tem na foto) pra bater o valor final.

Total states number:  48
Total demand:  1477026
Each state represents a demand center


In [5]:
demand_centers = [f"{s}" for s in state_demand.keys()]

pprint(demand_centers)

print(
    """
    The company has more than 70,000 individual customer accounts, and these are aggregated into 191 active demand centers.
    A demand center is a grouping of zip code areas into a zip sectional center as the focus of the collected demand.
    These demand centers, along with how they are currently being served, are given in Table 3
    """
)

['WA',
 'OR',
 'CA',
 'NV',
 'AZ',
 'ID',
 'UT',
 'MT',
 'ND',
 'WY',
 'CO',
 'SD',
 'NE',
 'KS',
 'MN',
 'IA',
 'MO',
 'NM',
 'TX',
 'OK',
 'AR',
 'LA',
 'WI',
 'IL',
 'MI',
 'IN',
 'KY',
 'OH',
 'ME',
 'NH',
 'RI',
 'NJ',
 'NY',
 'PA',
 'CT',
 'MA',
 'VA',
 'WV',
 'MD',
 'VT',
 'DE',
 'TN',
 'MS',
 'AL',
 'GA',
 'FL',
 'SC',
 'NC']

    The company has more than 70,000 individual customer accounts, and these are aggregated into 191 active demand centers.
    A demand center is a grouping of zip code areas into a zip sectional center as the focus of the collected demand.
    These demand centers, along with how they are currently being served, are given in Table 3
    


In [6]:
# 2.2 Plant current capacities C[p]
C = {
    "Covington, KY": 620_000,
    "New York, NY": 430_000,
    "Arlington, TX": 300_000,
    "Long Beach, CA": 280_000,
}

# Plant stocking capacities C'[p]
C_prime = {
    "Covington, KY": 450_000,
    "New York, NY": 380_000,
    "Arlington, TX": 140_000,
    "Long Beach, CA": 180_000,
}

In [7]:
# 2.3 Unit production cost v[p] (variable production cost)
rho = {
    "Covington, KY": 21.0,
    "New York, NY": 19.9,
    "Arlington, TX": 21.6,
    "Long Beach, CA": 21.1,
}

In [8]:
# 2.4 Distance matrices (in miles)
#     d_pw[p][w] = distance plant → warehouse
#     d_wj[w][d] = distance warehouse → demand center
#     d_pd[p][d] = distance plant → demand center

df_pw = pd.read_excel(
    "../distances/distance_matrix.xlsx", sheet_name="d_pw", index_col=0
)
df_ws = pd.read_excel(
    "../distances/distance_matrix.xlsx", sheet_name="d_ws", index_col=0
)
df_pd = pd.read_excel(
    "../distances/distance_matrix.xlsx", sheet_name="d_pd", index_col=0
)

d_pw = df_pw.to_dict()
d_wj = df_ws.to_dict()
d_pd = df_pd.to_dict()

In [9]:
print("Distance matrices:")
print("d_pw (plant to warehouse):")
pprint(d_pw)
print("d_wd (warehouse to demand center):")
pprint(d_wj)
print("d_pd (plant to demand center):")
pprint(d_pd)

Distance matrices:
d_pw (plant to warehouse):
{'Arlington, TX': {'Albuquerque': 687.6652353092529,
                   'Arlington': 0.0,
                   'Atlanta': 891.0323335828947,
                   'Baltimore': 1482.562875569311,
                   'Billings': 1307.792000488886,
                   'Boston': 1889.918800799371,
                   'Buffalo': 1463.237174923549,
                   'Camp Hill': 1486.478128223514,
                   'Chicago': 985.082896360298,
                   'Chicago *': 985.082896360298,
                   'Cleveland': 1254.780723551424,
                   'Columbus': 1120.254001730982,
                   'Covington': 1000.19101222514,
                   'Davenport': 849.1005447837437,
                   'Denver': 786.9697014998968,
                   'Detroit': 1222.588194270302,
                   'El Paso': 666.4948924864918,
                   'Grand Rapids': 1133.959741585327,
                   'Greensboro': 1222.089323934379,
              

In [10]:
# 2.5 Inbound cost c_in[p][w] = 0.92 + 0.0034*d_pw
c_in = {p: {w: 0.92 + 0.0034 * d_pw[p][w] for w in warehouses} for p in plants}

c_in

{'Covington, KY': {'Atlanta': 2.4272774329579416,
  'Boston': 3.9535914817634157,
  'Buffalo': 2.535070978525477,
  'Chicago': 1.9580823980844708,
  'Cleveland': 1.8349688507610071,
  'Davenport': 2.4030591642986976,
  'Detroit': 1.8905207382893088,
  'Grand Rapids': 2.0445698736760733,
  'Greensboro': 2.2793562728871803,
  'Kansas City': 3.147272161717205,
  'Baltimore': 2.657227925519008,
  'Memphis': 2.594734108340997,
  'Milwaukee': 2.2535432073390314,
  'Orlando': 3.9871711627998474,
  'Pittsburgh': 1.9763326218750703,
  'Portland': 9.05469775252212,
  'W Sacramento': 9.039676346863951,
  'W Chester': 2.93584517753538,
  'Albuquerque': 6.037042073785832,
  'Billings': 6.263933169543179,
  'Denver': 5.404801580177727,
  'El Paso': 6.3884120939698175,
  'Camp Hill': 2.6091414004110898,
  'Houston': 4.57445025793399,
  'Las Vegas': 7.826494158157176,
  'Minneapolis': 3.404887394002048,
  'New Orleans': 3.8068041455099473,
  'Phoenix': 7.396318248737397,
  'Richmond': 2.55366242025510

In [11]:
# Ler tabela 3 do excel
table3 = pd.read_excel(
    "../misc/tabelas.xlsx",
    sheet_name="table3",
)
table3.head()

,Warehouse No.,Storage ($/$),Handling ($/cwt),Stock Order Processing ($/order),Stock Order Size,Customer Order Processing ($/order),Customer Order size (cwt/order),Local delivery rate ($/cwt)
0,1,0.0672,0.46,18,400,1.79,9.05,1.90
1,2,0.0567,0.54,18,400,1.74,10.92,3.89
2,3,0.0755,0.38,18,400,2.71,11.59,2.02
3,4,0.0735,0.59,18,400,1.74,11.30,4.31
4,5,0.0946,0.50,18,401,0.83,9.31,1.89


In [12]:
# 2.6 Outbound cost c_out[w][j]:
local_rate = {i: v for i, v in enumerate(table3["Local delivery rate ($/cwt)"])}

c_out = {}
for i, w in enumerate(warehouses):
    c_out[w] = {}
    for j in demand_centers:
        d = d_wj[w][j]
        if d <= 30:
            # if d<=30: use local cartage rate from Table 3
            c_out[w][j] = local_rate[i]
        else:
            # else use 5.45 + 0.0037*d
            c_out[w][j] = 5.45 + 0.0037 * d

c_out

{'Atlanta': {'WA': 15.018734257545375,
  'OR': 15.086838655921529,
  'CA': 14.301949606240893,
  'NV': 13.476376340034427,
  'AZ': 12.502894241422542,
  'ID': 13.4610627329631,
  'UT': 12.5071509738893,
  'MT': 12.896401241660543,
  'ND': 10.90424941646361,
  'WY': 11.556809713385324,
  'CO': 10.885761021260969,
  'SD': 10.140755770107987,
  'NE': 9.328876085584593,
  'KS': 8.778909538485987,
  'MN': 9.586532010870588,
  'IA': 8.730608876849509,
  'MO': 7.878703630718229,
  'NM': 11.049832408864248,
  'TX': 8.933632630815582,
  'OK': 8.663801105303989,
  'AR': 7.614502874099189,
  'LA': 7.523916365336328,
  'WI': 8.713724162852385,
  'IL': 7.971137147534948,
  'MI': 8.253574815013415,
  'IN': 7.469105548404226,
  'KY': 6.72587140515843,
  'OH': 7.543632033898609,
  'ME': 10.209109252907348,
  'NH': 9.699540821506881,
  'RI': 9.447106234885357,
  'NJ': 8.648059394612009,
  'NY': 8.8193646671037,
  'PA': 8.200712899680434,
  'CT': 9.131001337788902,
  'MA': 9.556539138962261,
  'VA': 7.5

In [13]:
# custos de transporte da fabrica direto para o demand center.
c_out_prime = {}

for p in plants:
    c_out_prime[p] = {}
    for j in demand_centers:
        try:
            d = d_pd[p][j]
        except KeyError:
            print(f"{p} or {j} not found in d_pd")
            break
        if d <= 30:
            # if d<=30: use local cartage rate from Table 3
            # c_out_prime[p][j] = local_rate[p]
            c_out_prime[p][j] = 0

            # TODO: verify this.
        else:
            # else use 5.45 + 0.0037*d
            c_out_prime[p][j] = 5.45 + 0.0037 * d

In [14]:
# Quanto representa 1 cwt. em $? Dividimos as vendas totais ($) pela demand total (cwt)
Gamma = 160_000_000 / 1_477_026  # ($ / cwt)
Gamma

108.32578438023434

In [15]:
# Custo de estocagem do armazém w
tau = {}

for i, w in enumerate(warehouses):
    # Fiz o casting para evitar o np.float64 nos prints, mas nao precisava
    tau[w] = float(table3["Storage ($/$)"][i])

In [16]:
# Custo de handling do armazém w (epsilon)

epsilons = {}

for i, w in enumerate(warehouses):
    epsilons[w] = float(table3["Handling ($/cwt)"][i])

In [17]:
# gamma

gammas = {}

for i, w in enumerate(warehouses):
    gammas[w] = float(table3["Stock Order Processing ($/order)"][i])

In [18]:
# delta

deltas = {}

for i, w in enumerate(warehouses):
    deltas[w] = float(table3["Stock Order Size"][i])

In [19]:
# omega

omegas = {}

for i, w in enumerate(warehouses):
    omegas[w] = float(table3["Customer Order size (cwt/order)"][i])

In [20]:
# phi

phi = {}
for i, w in enumerate(warehouses):
    phi[w] = float(table3["Customer Order Processing ($/order)"][i])

In [21]:
# Big-M for linkage: no warehouse ships more than total demand
# M = sum(S.values())
# M

## 3. Defining the model

In [22]:
m = Model(name="UsemoreWarehousing")

Set parameter Username
Set parameter LicenseID to value 2602866
Academic license - for non-commercial use only - expires 2025-12-22


### Decision variables

In [23]:
Z = m.addVars(warehouses, vtype=GRB.BINARY, name="z")
X = m.addVars(plants, warehouses, vtype=GRB.CONTINUOUS, name="x")
Y = m.addVars(warehouses, demand_centers, vtype=GRB.CONTINUOUS, name="y")
W = m.addVars(plants, demand_centers, vtype=GRB.CONTINUOUS, name="w")
# U = m.addVars(plants, demand_centers, vtype=GRB.BINARY, name="u")

## 4. Objective Function

### Define all the costs components

In [24]:
# inbound transport (from plants to warehouses)
inbound__transport_cost = gurobipy.quicksum(
    c_in[i][j] * X[i, j] for i in plants for j in warehouses
)

# outbound transport (from warehouses to demand nodes)
outbound_transport_cost = gurobipy.quicksum(
    c_out[j][k] * Y[j, k] for j in warehouses for k in demand_centers
)

# transport cost from plants to demand centers
direct_transport_cost = gurobipy.quicksum(
    c_out_prime[i][k] * W[i, k] for i in plants for k in demand_centers
)

# production cost
production_costs = gurobipy.quicksum(
    rho[p] * (X[p, w] + W[p, d])
    for p in plants
    for w in warehouses
    for d in demand_centers
)

# storage cost in warehouses
storage_costs = gurobipy.quicksum(
    X[p, w] * Gamma * tau[w] for p in plants for w in warehouses
)

# custo de handling nos armazéns
handling_costs = gurobipy.quicksum(
    2 * Z[w] * epsilons[w] * X[p, w] for p in plants for w in warehouses
)

# custo de processamento do pedido de estoque
stock_order_processing_costs = gurobipy.quicksum(
    X[p, w] * (gammas[w] / deltas[w]) * Z[w] for p in plants for w in warehouses
)

# custo de processamento do pedido do cliente
customer_order_processing_costs = gurobipy.quicksum(
    Y[w, d] * (phi[w] / omegas[w]) * Z[w] for w in warehouses for d in demand_centers
)

#### Define the objective function

In [25]:
m.setObjective(
    inbound__transport_cost
    + outbound_transport_cost
    + direct_transport_cost
    + production_costs
    + storage_costs
    + handling_costs
    + stock_order_processing_costs
    + customer_order_processing_costs,
    sense=GRB.MINIMIZE,
)

## Constraints

In [26]:
# Demand satisfaction
# m.addConstrs((Y.sum("*", d) == S[d] for d in demand_centers), "demand")

for d in demand_centers:
    m.addConstr(
        gurobipy.quicksum(Y[w, d] for w in warehouses)
        + gurobipy.quicksum(W[p, d] for p in plants)
        == S[d],
        name=f"demand_{d}",
    )

In [27]:
# Plant capacity
# m.addConstrs((X.sum(p, "*") <= C[p] for p in plants), "plantCap")

for p in plants:
    m.addConstr(
        gurobipy.quicksum(X[p, w] for w in warehouses)
        + gurobipy.quicksum(W[p, d] for d in demand_centers)
        <= C[p],
        name=f"plantCap_{p}",
    )

In [28]:
# restrição dos 10400

for w in warehouses:
    m.addConstr(
        gurobipy.quicksum(Y[w, d] for d in demand_centers) >= 10_400 * Z[w],
        name=f"minThroughput_{w}",
    )

In [29]:
# restrições do u_{pd}

for p in plants:
    m.addConstr(
        # gurobipy.quicksum(U[p, d] * W[p, d] for d in demand_centers) <= C_prime[p],
        gurobipy.quicksum(W[p, d] for d in demand_centers) <= C_prime[p],
    )

# for p in plants:
#     for d in demand_centers:
#         # if U=1 ⇒ W ≤ 100
#         m.addGenConstrIndicator(U[p,d], True,  W[p,d] <= 100,
#                                 name=f"ind1_{p}_{d}")
#         # if U=0 ⇒ W ≥ 100+ε  (ε small, to keep the logical ⇔ strict)
#         m.addGenConstrIndicator(U[p,d], False, W[p,d] >= 100 + 1e-6,
#                                 name=f"ind0_{p}_{d}")

In [30]:
# Flow balance at each warehouse
# m.addConstrs(( x.sum('*',w) == y.sum(w,'*') for w in warehouses ), "flowBalance")

for w in warehouses:
    m.addConstr(
        gurobipy.quicksum(X[p, w] for p in plants)
        == gurobipy.quicksum(Y[w, d] for d in demand_centers),
        name=f"flowBalance_{w}",
    )

## Solver

In [31]:
m.params.TimeLimit = 300  # seconds

m.optimize()

Set parameter TimeLimit to value 300
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Arch Linux")

CPU model: Intel(R) Core(TM) i5-10300H CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
TimeLimit  300

Optimize a model with 144 rows, 2524 columns and 7308 nonzeros
Model fingerprint: 0xa062ce04
Model has 2288 quadratic objective terms
Variable types: 2480 continuous, 44 integer (44 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [8e-01, 1e+03]
  QObjective range [2e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+03, 6e+05]
Found heuristic solution: objective 1.403517e+09
Presolve removed 44 rows and 44 columns
Presolve time: 0.00s
Presolved: 100 rows, 2480 columns, 5152 nonzeros
Variable types: 2480 continuous, 0 integer (0 binary)

Root relaxation: objective 1.389203e+09, 262 iterations, 0.00 seconds (0.01 work units)

In [32]:
m.printStats()

Statistics for model 'UsemoreWarehousing':
  Problem type                : MIQP
  Linear constraint matrix    : 144 rows, 2524 columns, 7308 nonzeros
  Variable types              : 2480 continuous, 44 integer (44 binary)
  Matrix range                : [1e+00, 1e+04]
  Objective range             : [8e-01, 1e+03]
  QObjective range            : [2e-01, 2e+01]
  Bounds range                : [1e+00, 1e+00]
  RHS range                   : [1e+03, 6e+05]


In [33]:
m.printQuality()

Solution quality statistics for model 'UsemoreWarehousing' :
  Maximum violation:
    Bound       : 0.00000000e+00
    Constraint  : 0.00000000e+00
    Integrality : 0.00000000e+00


## Visualize results

In [34]:
print("\n--- Warehouse openings ---")
for w in warehouses:
    if Z[w].X > 0.1:
        print(f" Open warehouse at {w}")


--- Warehouse openings ---


In [35]:
print("\n--- Plant → Warehouse flows ---")
for p, w in X.keys():
    if X[p, w].X > 1e-6:
        print(f" {p} → {w}: {X[p, w].X:.0f} cwt")


--- Plant → Warehouse flows ---
 Covington, KY → Atlanta: 95746 cwt
 Covington, KY → Chicago: 74254 cwt
 New York, NY → Boston: 50000 cwt
 Arlington, TX → Houston: 7026 cwt
 Long Beach, CA → Houston: 7947 cwt
 Long Beach, CA → Las Vegas: 86420 cwt
 Long Beach, CA → Minneapolis: 5633 cwt


In [36]:
print("\n--- Plant → Demand flows ---")
for w, j in W.keys():
    if W[w, j].X > 1e-6:
        print(f" {w:<13} → {j}: {W[w, j].X:.0f} cwt")


--- Plant → Demand flows ---
 Covington, KY → MO: 31145 cwt
 Covington, KY → IL: 68208 cwt
 Covington, KY → MI: 105181 cwt
 Covington, KY → IN: 43994 cwt
 Covington, KY → KY: 3870 cwt
 Covington, KY → OH: 155123 cwt
 Covington, KY → TN: 42479 cwt
 New York, NY  → ME: 2916 cwt
 New York, NY  → NH: 4546 cwt
 New York, NY  → RI: 17000 cwt
 New York, NY  → NJ: 21154 cwt
 New York, NY  → NY: 160917 cwt
 New York, NY  → PA: 65108 cwt
 New York, NY  → CT: 26187 cwt
 New York, NY  → VA: 17667 cwt
 New York, NY  → WV: 9168 cwt
 New York, NY  → MD: 19284 cwt
 New York, NY  → VT: 2928 cwt
 New York, NY  → DE: 3044 cwt
 New York, NY  → SC: 1733 cwt
 New York, NY  → NC: 28348 cwt
 Arlington, TX → SD: 1049 cwt
 Arlington, TX → NE: 7347 cwt
 Arlington, TX → KS: 6961 cwt
 Arlington, TX → MO: 10535 cwt
 Arlington, TX → TX: 80438 cwt
 Arlington, TX → OK: 13517 cwt
 Arlington, TX → AR: 4910 cwt
 Arlington, TX → LA: 38 cwt
 Arlington, TX → MS: 15205 cwt
 Long Beach, CA → WA: 4456 cwt
 Long Beach, CA → OR

In [37]:
print("\n--- Warehouse → Demand flows ---")
for w, j in Y.keys():
    if Y[w, j].X > 1e-6:
        print(f" {w:<13} → {j}: {Y[w, j].X:.0f} cwt")


--- Warehouse → Demand flows ---
 Atlanta       → AL: 15835 cwt
 Atlanta       → GA: 29559 cwt
 Atlanta       → FL: 46405 cwt
 Atlanta       → SC: 3947 cwt
 Boston        → ME: 12913 cwt
 Boston        → MA: 37087 cwt
 Chicago       → IA: 32175 cwt
 Chicago       → WI: 37448 cwt
 Chicago       → IL: 4631 cwt
 Houston       → LA: 14973 cwt
 Las Vegas     → WA: 27981 cwt
 Las Vegas     → NV: 16755 cwt
 Las Vegas     → ID: 7153 cwt
 Las Vegas     → UT: 9001 cwt
 Las Vegas     → MT: 4140 cwt
 Las Vegas     → ND: 5703 cwt
 Las Vegas     → WY: 1004 cwt
 Las Vegas     → CO: 11147 cwt
 Las Vegas     → NM: 3536 cwt
 Minneapolis   → MN: 5633 cwt
